In [1]:
import copy
import glob
import os
import time
from collections import deque

import gym
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from a2c_ppo_acktr import algo, utils
from a2c_ppo_acktr.algo import gail
from a2c_ppo_acktr.arguments import get_args
from a2c_ppo_acktr.envs import make_vec_envs
from a2c_ppo_acktr.model import Policy, CNNBase
from a2c_ppo_acktr.storage import RolloutStorage
from evaluation import evaluate

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [2]:
import gym
import gym_interf
import numpy as np
import torch

from baselines.common.vec_env.shmem_vec_env import ShmemVecEnv
from baselines.common.vec_env.subproc_vec_env import SubprocVecEnv
from baselines.common.vec_env.dummy_vec_env import DummyVecEnv
from a2c_ppo_acktr.envs import VecPyTorch

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
N_ENVS = 8
N_STEPS = 128

class TimeLimitWrapper(gym.Wrapper):
    def step(self, action):
        obs, rew, done, info = self.env.step(action)
        if done and 200 == self.env.n_steps:
            info['bad_transition'] = True

        return obs, rew, done, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)



def make_interf_env(seed):
    env = gym.make('interf-v1')
    env.set_calc_reward('delta_visib')
    #env.set_calc_image('gpu')
    env.seed(seed)
    return env


temp_env = make_interf_env(0)
obs = temp_env.reset()
ACTION_SPACE = temp_env.action_space
print(ACTION_SPACE)
N_ACTIONS = ACTION_SPACE.n
OBS_SHAPE = obs.shape


env_lambda = [
    lambda env_seed=env_seed: make_interf_env(seed=env_seed)
    for env_seed in range(N_ENVS)]

envs = SubprocVecEnv(env_lambda, context='fork')
#envs = ShmemVecEnv(env_lambda, context='fork')
envs = VecPyTorch(envs, DEVICE)

print(envs.reset().shape)
print(envs.reset().type)




print(DEVICE)
print(obs.dtype, obs.shape, N_ACTIONS)
assert obs.shape == (16, 64, 64), obs.shape
#assert obs.dtype == torch.uint8



Discrete(8)
torch.Size([8, 16, 64, 64])
<built-in method type of Tensor object at 0x7f8609adea20>
cuda
uint8 (16, 64, 64) 8


/home/dmitry/dev/gym_interf/gym_interf/envs/interf_env.py:260: RuntimeWarning: divide by zero encountered in double_scalars
  band_width_y = InterfEnv.lamb / abs(wave_vector2[1])
/home/dmitry/dev/gym_interf/gym_interf/envs/interf_env.py:260: RuntimeWarning: divide by zero encountered in double_scalars
  band_width_y = InterfEnv.lamb / abs(wave_vector2[1])
/home/dmitry/dev/gym_interf/gym_interf/envs/interf_env.py:260: RuntimeWarning: divide by zero encountered in double_scalars
  band_width_y = InterfEnv.lamb / abs(wave_vector2[1])
/home/dmitry/dev/gym_interf/gym_interf/envs/interf_env.py:260: RuntimeWarning: divide by zero encountered in double_scalars
  band_width_y = InterfEnv.lamb / abs(wave_vector2[1])
/home/dmitry/dev/gym_interf/gym_interf/envs/interf_env.py:260: RuntimeWarning: divide by zero encountered in double_scalars
  band_width_y = InterfEnv.lamb / abs(wave_vector2[1])
/home/dmitry/dev/gym_interf/gym_interf/envs/interf_env.py:260: RuntimeWarning: divide by zero encountered

In [3]:
print(OBS_SHAPE)
print(ACTION_SPACE)

actor_critic = Policy(
    OBS_SHAPE,
    ACTION_SPACE).to(DEVICE)

agent = algo.PPO(
    actor_critic,
    clip_param=0.1,
    ppo_epoch=4,
    num_mini_batch=4,
    value_loss_coef=0.5,
    entropy_coef=0.01,
    lr=2.5e-4,
    eps=1e-5,
    max_grad_norm=0.5)

print(actor_critic.recurrent_hidden_state_size)


rollouts = RolloutStorage(
    N_STEPS, N_ENVS,
    envs.observation_space.shape, envs.action_space,
    actor_critic.recurrent_hidden_state_size
)


(16, 64, 64)
Discrete(8)
1


In [4]:
from tqdm import trange

def evaluate_(actor_critic, env, n_games=10):
    n_solved = 0
    n_steps = 0
    dist = 0
    angle = 0
    rewards = 0
    
    eval_recurrent_hidden_states = torch.zeros(
        1, actor_critic.recurrent_hidden_state_size, device=DEVICE)
    eval_masks = torch.zeros(1, 1, device=DEVICE)
    
    for i in range(n_games):
        s = env.reset()
        istep = 0
        while(True):
            s = torch.FloatTensor(s.reshape(1, 16, 64, 64)).to(DEVICE)
            with torch.no_grad():
                _, action, _, eval_recurrent_hidden_states = actor_critic.act(
                    s,
                    eval_recurrent_hidden_states,
                    eval_masks,
                    deterministic=True)
            
            s, reward, done, info = env.step(action)
            istep += 1
            rewards += reward
            if done:
                n_solved += istep < 200
                n_steps += istep
                dist += info['dist']
                angle += info['angle_between_beams']

                break
    return n_solved / n_games, n_steps / n_games, dist / n_games, angle / n_games, rewards / n_games

In [5]:
evaluate_(actor_critic, make_interf_env(123))

(0.1, 180.2, 0.41105227542112405, 0.003988663848542675, -0.20612292293909457)

In [ ]:
from tqdm import trange
from tensorboardX import SummaryWriter

SAVE_INTERVAL = 10
LOG_INTERVAL = 10
EVAL_INTERVAL = 10

obs = envs.reset()
rollouts.obs[0].copy_(obs)
rollouts.to(DEVICE)


writer = SummaryWriter(
    '../learning_logs/ppo',
)


entropis = []
vlosses = []
plosses = []

rewards = np.zeros(N_ENVS, dtype=float)
dones = np.zeros(N_ENVS, dtype=float)
steps = np.zeros(N_ENVS, dtype=float)


start = time.time()
n_frames = int(1e7)
for j in trange(0, n_frames, N_ENVS * N_STEPS):
    
    # decrease learning rate linearly
    utils.update_linear_schedule(
        agent.optimizer, j, n_frames,
        2.5e-4)
    
    
    for step in range(N_STEPS):
        # Sample actions
        with torch.no_grad():
            value, action, action_log_prob, recurrent_hidden_states = actor_critic.act(
                rollouts.obs[step], rollouts.recurrent_hidden_states[step],
                rollouts.masks[step])

        # Obser reward and next obs
        obs, reward, done, infos = envs.step(action)
        rewards += torch.tensor(reward).numpy().reshape(N_ENVS)
        dones += torch.tensor(done).numpy().reshape(N_ENVS)
        steps += 1


        # If done then clean the history of observations.
        masks = torch.FloatTensor(
            [[0.0] if done_ else [1.0] for done_ in done])
        bad_masks = torch.FloatTensor(
            [[0.0] if 'bad_transition' in info.keys() else [1.0]
                for info in infos])
        rollouts.insert(obs, recurrent_hidden_states, action,
                        action_log_prob, value, reward, masks, bad_masks)

    with torch.no_grad():
        next_value = actor_critic.get_value(
            rollouts.obs[-1], rollouts.recurrent_hidden_states[-1],
            rollouts.masks[-1]).detach()

    rollouts.compute_returns(
        next_value, use_gae=True, gamma=0.99,
        gae_lambda=0.95, use_proper_time_limits=True)

    value_loss, action_loss, dist_entropy = agent.update(rollouts)
    vlosses.append(value_loss)
    plosses.append(action_loss)
    entropis.append(dist_entropy)

    rollouts.after_update()

    # save for every interval-th episode or for the last epoch
    if np.sum(dones) >= 100:
        save_path = os.path.join('logs', 'ppo')
        try:
            os.makedirs(save_path)
        except OSError:
            pass

        torch.save([
            actor_critic,
            getattr(utils.get_vec_normalize(envs), 'ob_rms', None)
        ], os.path.join(save_path, 'interf' + ".pt"))


    if np.sum(dones) >= 100:
        writer.add_scalar('entropy', np.mean(entropis), j)
        writer.add_scalar('vloss', np.mean(vlosses), j)
        writer.add_scalar('ploss', np.mean(plosses), j)
        writer.add_scalar('reward', np.mean(rewards / dones), j)
        writer.add_scalar('steps', np.mean(steps / dones), j)
        
        n_solved, n_steps, dist, angle, rewards = evaluate_(actor_critic, make_interf_env(j))
        print(n_solved, n_steps, dist, angle, rewards)
        writer.add_scalar('eval_solved_games', n_solved, j)
        writer.add_scalar('eval_steps', n_steps, j)
        writer.add_scalar('dist_between_beams', dist, j)
        writer.add_scalar('angle_between_beams', angle, j)
        writer.add_scalar('eval_reward', rewards, j)

        entropis = []
        vlosses = []
        plosses = []
        rewards = np.zeros(N_ENVS, dtype=float)
        dones = np.zeros(N_ENVS, dtype=float)
        steps = np.zeros(N_ENVS, dtype=float)

  0%|          | 21/9766 [00:35<9:48:54,  3.63s/it]

0.1 180.4 0.12381757753980054 4.4224657174857564e-05 0.2888411076127483


  0%|          | 39/9766 [01:09<10:40:39,  3.95s/it]

0.0 200.0 0.11611317428221254 3.2164296943903235e-05 0.3599107567422558


  1%|          | 57/9766 [01:43<10:07:18,  3.75s/it]

...

0.6 87.4 0.01256966517466268 0.0 0.4446886287310301


 59%|█████▉    | 5761/9766 [5:11:09<4:38:13,  4.17s/it]

0.7 68.0 0.02320387844070991 1.4901161193847657e-09 0.3123484245488563


 59%|█████▉    | 5763/9766 [5:11:16<4:18:54,  3.88s/it]

0.8 47.4 0.017196318053658403 2.9802322387695314e-09 0.41340809143559387
